# Travelling Salesman Problem using Google Maps API

### 1. Create namedtuple for the Route details
start = postal code of origin<br>
end = postal code of destination<br>
distance = total distance from start to end (in meter)<br>
steps = detailed steps from one point to next point and the distance of each step (will used to create PolyLine on the map)<br>
orig_lat = origin latitude<br>
orig_lng = origin longitude<br>
dest_lat = destination latitude<br>
dest_lng = destination longitude

In [244]:
import pandas as pd
import itertools
import numpy as np
import math
import random
import googlemaps
from datetime import datetime
from collections import namedtuple

gmaps = googlemaps.Client(key='Enter Your Google Maps API Key')

#create a named tuple for the routes
Route=namedtuple('Route',['start','end','distance','steps','orig_lat','orig_lng','dest_lat','dest_lng'])

### 2. 'make_route' functions to return the 'Route' namedtuple

In [132]:
#function to make the 'Route' namedtuple
def make_route(start,end,distance,steps,orig_lat,orig_lng,dest_lat,dest_lng):
    return Route(start,end,distance,steps,orig_lat,orig_lng,dest_lat,dest_lng)

### 3. This is how the API output looks like
Different types of mode of transport: 'driving', 'walking', 'bicycling', 'transit'

<img src="gmaps_api_result.png">

### 4. 'get_directions' function to return the API results. This will be used as inputs to 'make_route' function

In [131]:
#function to return the API results
def get_directions(origin,dest):
    directions_result = gmaps.directions(origin,
                                     dest,
                                     mode="driving",
                                     departure_time=datetime.now())
    distance=directions_result[0]['legs'][0]['distance']['value'] #total distance from origin to destination
    orig_lat=directions_result[0]['legs'][0]['start_location']['lat'] #origin latitude
    orig_lng=directions_result[0]['legs'][0]['start_location']['lng'] #origin longitude
    dest_lat=directions_result[0]['legs'][0]['end_location']['lat'] #destination latitude
    dest_lng=directions_result[0]['legs'][0]['end_location']['lng'] #destination longitude
    
    #'steps' to be returned as a list that contains two coordinates, point 1 and point 2, and also the distance (in meter) between these two points
    steps=[]
    for i in range(len(directions_result[0]['legs'][0]['steps'])):
        start_pos=[directions_result[0]['legs'][0]['steps'][i]['start_location']['lat'],directions_result[0]['legs'][0]['steps'][i]['start_location']['lng']]
        end_pos=[directions_result[0]['legs'][0]['steps'][i]['end_location']['lat'],directions_result[0]['legs'][0]['steps'][i]['end_location']['lng']]
        step_pos=[start_pos,end_pos] #two coordinates of point 1 and point 2
        step_distance=directions_result[0]['legs'][0]['steps'][i]['distance']['value'] #distance between the two points
        steps.append((step_pos,step_distance)) #append to the 'steps' list
        
    return origin,dest,distance,steps,orig_lat,orig_lng,dest_lat,dest_lng
    

### 5. Construct a 'Graph' class with 'address' and 'routes' as the class attributes
* The 'address' will be in a list of postal codes. 'Singapore ' is added in front of each address to make sure that Google Maps API will return the correct locations in Singapore

* Get all the possible routes between 2 points where **order matters** (a.k.a permutations) using itertools.permutations. 
Number of permutations is:

 
$$nPk=\frac{n!}{(n-k)!}$$


* 'routes' will be list of namedtuple 'Route' for all the possible permutations.
* If the users enter new list of postal codes ('address'), 'add_route' function will call Google Maps API only for the new addresses that are not yet in 'routes' and append the new routes to existing 'routes'. 
* 'totaldistancetour' function is to calculate the total distance of a given 'tour' <br>
>**example:** <br>
'tour' is given as below:<br>
a -> b -> c -> d -> a<br><br>
the function will return total distance that is the sum of the following:<br>
distance from a -> b<br>
distance from b -> c<br>
distance from c -> d<br>
distance from d -> a (return back to origin)

### 6. Brute Force Method
Loop through all the possible tours and get the total distance. The best tour is the one with the shortest total distance.

### 7. Simulated Annealing Method
* It is inspired by annealing process in metal work. Temperature of solid metal is increased until it melts then cooled carefully.

#### Algorithm
1. Assign a high initial value of temperature then slowly decrease.
2. It will more frequently accept solutions which are worse than our current solution as long as temperature variable is still high.
3. By doing this, we allow the algorithm to jump out of any local optimum.
4. The chance of accepting worse solutions will reduce as the temperature is reduced.
5. It will slowly allow the algorithm to focus on area of the search space in which a close to optimum solution can be found.

#### How to decide which solutions to accept
1. If the neighbour solution is better than our current solution, accept it.
2. If the neighbour solution is worse, consider these 2 factors: how much worse it is and how high the current temperature is.


$$P = exp (\frac{currentsolution - neigbour}{temperature})$$

P will be accepted if it is greater than random.random() that will return any random float between 0.0 and 1.0. 
Therefore, the smaller the change between current and neighbour & the higher the temperature, more likely it is to accept the solution.
<br><br>
Since simulated annealing algorithm only returns the value close to minimum solution, it is better to repeat the process several times and compare the results to find the best solution.

In [238]:
class Graph:
    def __init__(self,address):
        #add 'Singapore' to the postal code
        self.address=['Singapore '+a for a in address]
        
        #find all the possible route permutations
        combination=list(itertools.permutations(self.address,2))
        
        #make routes that consist of start, end, distance
        self.routes=[make_route(i[0],i[1],get_directions(i[0],i[1])[2],get_directions(i[0],i[1])[3],
                               get_directions(i[0],i[1])[4],get_directions(i[0],i[1])[5],
                               get_directions(i[0],i[1])[6],get_directions(i[0],i[1])[7]) for i in combination]
    
    #function to add new routes and append them to self.routes
    def add_route(self,address):
        self.address=['Singapore '+a for a in address]
        combination=list(itertools.permutations(self.address,2))
        for new_route in combination:
            if new_route in [(route.start,route.end) for route in self.routes]:
                pass
            else:
                self.routes.append(Route(new_route[0],new_route[1],
                                         get_directions(new_route[0],new_route[1])[2],get_directions(new_route[0],new_route[1])[3],
                                         get_directions(new_route[0],new_route[1])[4],get_directions(new_route[0],new_route[1])[5],
                                         get_directions(new_route[0],new_route[1])[6],get_directions(new_route[0],new_route[1])[7]))
    
    #find the total distance for specific tour
    def totaldistancetour(self,tour):
        d=0
        for i in range(1,len(tour)):
            origin=self.test_address[tour[i-1]]
            dest=self.test_address[tour[i]]
            distance=[route.distance for route in self.routes if (route.start==origin and route.end==dest)][0]
            d=d+distance
        origin=self.test_address[tour[len(tour)-1]]
        dest=self.test_address[tour[0]]
        distance=[route.distance for route in self.routes if (route.start==origin and route.end==dest)][0]
        d=d+distance
        return d
    
    #brute force method to find the shortest tour
    def tsp_brute_force_gmaps(self,test_address):
        '''
        this is brute force method to find the best tour with the shortest total distance. 
        It finds all the possible tours and find the one with the shortest distance.
        '''
        
        #to add the tested routes to self.routes if they do not exist
        self.add_route(test_address)
        self.test_address=['Singapore '+a for a in test_address]
        n=len(self.test_address)
        
        #get a randomized n-length list of integers between 0 and (n-1).
        tour=random.sample(range(n),n)
        
        #get all the possible tours using permutations
        allpossibletour=list(itertools.permutations(tour))
        
        #initialize the best distance=1000000, we will need to find a distance shorter than this
        lbest=1000000
        
        #initialize the best tour 
        ibest=tour
        
        #find the total distance for each possible tour and get the shortest one
        for i in range(len(allpossibletour)):
            l=self.totaldistancetour(allpossibletour[i])
            if l < lbest:
                lbest=l
                ibest=allpossibletour[i]                
        
        return ibest,lbest
    
    #using simulated annealing 
    def tsp_anneal_gmaps(self,test_address):
        self.test_address=['Singapore '+ a for a in test_address]
        n=len(self.test_address)
        tour=random.sample(range(n),n)
        for temperature in np.logspace(0,5,num=10000)[::-1]:
            oldDistance=self.totaldistancetour(tour)
            
            #get two random indexes to swap with each other
            [i,j]=sorted(random.sample(range(n),2)) 
            
            #do the swap between the two random indexes
            newTour=tour[:i]+tour[j:j+1]+tour[i+1:j]+tour[i:i+1] + tour[j+1:]
            
            #find the total distance of the new tour
            newDistance=self.totaldistancetour(newTour)
            
            #if it meets the equation, the new tour will be accepted
            if math.exp((oldDistance-newDistance)/temperature)>random.random():
                tour=newTour.copy()
            return tour,self.totaldistancetour(tour)
    
    def repeated_tsp_anneal(self,test_address):
        picked_route=''
        best_distance=1000000
        
        #repeat anneal algorithm for 100 times to find the best solution
        for i in range(100):
            distance=self.tsp_anneal_gmaps(address)[1]
            if distance<best_distance:
                best_distance=distance
                picked_route=self.tsp_anneal_gmaps(address)[0]
        return picked_route,distance

### 8. Get the best tour/route and find the steps to be taken when going to one point to the next point. This is used to create PolyLine on the map.

In [242]:
def get_steps(instance_name,method):
    steps=[]
    routes=instance_name.routes
    if method=='brute_force':
        best_route=instance_name.tsp_brute_force_gmaps(address)
        print(best_route)
    else:
        best_route=instance_name.repeated_tsp_anneal(address)
        print(best_route)
    for i in range(1,len(best_route[0])):
        origin='Singapore '+address[best_route[0][i-1]]
        dest='Singapore '+address[best_route[0][i]]
        
        #get the steps when going to one point to the next and append it to 'steps' list
        step=[route.steps for route in routes if (route.start==origin and route.end==dest)]
        steps.append(step)
    
    #get the steps from the destination to the origin
    origin='Singapore '+address[best_route[0][len(best_route[0])-1]]
    dest='Singapore '+address[best_route[0][0]]
    step=[route.steps for route in routes if (route.start==origin and route.end==dest)]
    steps.append(step)
    return steps,best_route
    #return [s[i][0][j][0] for i in range(len(s)) for j in range(len(s[i][0]))]


### 9. Create a function to return the coordinates based on the address (postal code) as the input

In [246]:
#create function to return the coordinates of an address (postal code)
def get_lat_lng(postal_code,route):
    for route in route.routes:
        if postal_code in route.start:
            pos=route.orig_lat,route.orig_lng
            break
    return pos

### 10. Find the longest sub-step for each step. This will be the line where arrow will be drawn.

In [249]:
def get_arrow_point(steps):
    max_route=[]
    for i in range(len(steps)):
        step_distance=[]
        for j in range(len(steps[i][0])):
            step_distance.append(steps[i][0][j][1])
        #find the index of step with the longest distance
        max_index=[a for a, b in enumerate(step_distance) if b == max(step_distance)][0]
        max_route.append(steps[i][0][max_index][0])
    return max_route

### 11. Functions to calculate the bearing between two different points. This is used to draw the arrow.
'get_bearing' and 'get_arrows' functions from this article: https://medium.com/@bobhaffner/folium-lines-with-arrows-25a0fe88e4e

In [299]:
def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Link to the article: https://medium.com/@bobhaffner/folium-lines-with-arrows-25a0fe88e4e
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

In [300]:
def get_arrows(locations, color='blue', size=6, n_arrows=3):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    
    Note
    Link to the article: https://medium.com/@bobhaffner/folium-lines-with-arrows-25a0fe88e4e
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=rotation).add_to(map))
    return arrows

### 12. Create a function to visualize the route in map

In [311]:
import folium
from folium.features import DivIcon

def visualize_route(method,address,route):
    #create a new map
    map = folium.Map(location=get_lat_lng(address[1],route),tiles="cartodbpositron",zoom_start=12,width=1200,height=600)

    #add markers on the folium map
    for i in range(0,len(address)):
        marker=folium.Marker(location=[get_lat_lng(address[i],route)[0],get_lat_lng(address[i],route)[1]],
                             popup=address[i]
                            ).add_to(map)
     
    get_steps_result=get_steps(route,method)
    #get the steps and step distance of the best_route
    steps=get_steps_result[0]
    steps=[steps[i][0][j][0] for i in range(len(steps)) for j in range(len(steps[i][0]))]
    
    #get the best route and total distance
    best_route=get_steps_result[1]
    
    #draw the PolyLine for each step
    folium.PolyLine(steps,color="red", weight=2.5, opacity=1).add_to(map)
    
    #draw arrow for the longest step distance
    for arrow_point in get_arrow_point(get_steps_result[0]):
        get_arrows(locations=arrow_point, n_arrows=1)[0].add_to(map)
    
    #add numbers to show the sequence of the route
    for i in range(1,len(best_route[0])+1):
        point=get_lat_lng(address[best_route[0][i-1]],route)
        folium.map.Marker(
            [point[0],point[1]],
            icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(0,65),
                html='<div style="font-size: 12pt">'+str(i)+'</div>',
                )
            ).add_to(map)

    display(map)

In [227]:
address=['596569',
         '118733',
         '456532',
         '380119']

route=Graph(address)

In [312]:
visualize_route('anneal',address,route)

([3, 0, 1, 2], 54919)


Sample result:
<img src="sample_map_result.png">